# Test SINDy on Lorenz System

In [1]:
from functools import partial
import numpy as np
import pandas as pd
from scipy.integrate import odeint

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, MultiTaskLassoCV
from sklearn.feature_selection import SelectFromModel

from lorenz import lorenz_odes, lorenz_odes_vectorized
from sindy import polynomial_features, sparsify_dynamics_lstsq

## Simulate Lorenz system

In [20]:
dt = 0.01
T = 50
t = np.arange(dt, T + dt, dt)

In [21]:
# Lorenz system parameters
beta = 8 / 3
sigma = 10
rho = 28
n = 3

In [22]:
# Initial condition
x0 = (-8, 8, 27)

In [23]:
# Simulate using scipy.integrate.odeint method
# Produces same results as Matlab
fun = partial(lorenz_odes, sigma=sigma, beta=beta, rho=rho)
rtol = 10e-12
atol = 10e-12 * np.ones_like(x0)
x = odeint(fun, x0, t, tfirst=True, rtol=rtol, atol=atol)
assert x.shape == (5000, 3)

In [24]:
# Calculate derivatives (vectorized version)
dx = lorenz_odes_vectorized(0, x.T, sigma, beta, rho).T
assert dx.shape == (5000, 3)

## Identify system dynamics using sparse regression

In [7]:
lamb = 0.025  # sparsification knob lambda

### (i) Using ordinary least-squares

In [8]:
theta = polynomial_features(x, order=3)
assert theta.shape == (5000, 20)

In [9]:
# Estimate sparse dynamic model
xi = sparsify_dynamics_lstsq(theta, dx, lamb, n)
assert xi.shape == (20, 3)

In [10]:
xi

array([[  0.        ,   0.        ,   0.        ],
       [-10.        ,  28.        ,   0.        ],
       [ 10.        ,  -1.        ,   0.        ],
       [  0.        ,   0.        ,  -2.66666667],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   1.        ],
       [  0.        ,  -1.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.     

### (ii) Using L1-based feature selection in Scikit-Learn

See: https://scikit-learn.org/stable/modules/feature_selection.html#l1-based-feature-selection

In [11]:
poly = PolynomialFeatures(3)
theta = poly.fit_transform(x)
assert theta.shape == (5000, 20)
X, y = theta, dx

In [12]:
# Set L1 penalty
alpha = 2  # Above 1 works best

# Specify how many features to use
max_features = 7

# Find sparse model
estimator = Lasso(normalize=True, alpha=alpha)
selector = SelectFromModel(estimator, threshold=-np.inf, 
                           max_features=max_features)
selector.fit(X, y)

X_transform = selector.transform(X)
n_features = selector.transform(X).shape[1]
n_features

7

In [13]:
# Fit sparse model
estimator = LinearRegression()
estimator.fit(X_transform, y)
estimator.coef_.T.round(3)

array([[  0.   ,   0.   ,   0.   ],
       [-10.   ,  28.   ,  -0.   ],
       [ 10.   ,  -1.   ,  -0.   ],
       [ -0.   ,   0.   ,  -2.667],
       [  0.   ,  -0.   ,  -0.   ],
       [ -0.   ,   0.   ,   1.   ],
       [ -0.   ,  -1.   ,   0.   ]])

### (iii) With cross-validation

In [14]:
# Set L1 penalty
alpha = 2

# Specify how many features to use
max_features = 7

estimator = MultiTaskLassoCV(normalize=True, cv=5, alphas=[alpha]*4)

# Set minimum threshold
selector = SelectFromModel(estimator, threshold=-np.inf, 
                           max_features=max_features)
selector.fit(X, y)

X_transform = selector.transform(X)
n_features = selector.transform(X).shape[1]
n_features

7

In [15]:
estimator = LinearRegression()
estimator.fit(X_transform, y)
estimator.coef_.T.round(3)

array([[  0.   ,   0.   ,   0.   ],
       [-10.   ,  28.   ,  -0.   ],
       [ 10.   ,  -1.   ,  -0.   ],
       [ -0.   ,   0.   ,  -2.667],
       [  0.   ,  -0.   ,  -0.   ],
       [ -0.   ,   0.   ,   1.   ],
       [ -0.   ,  -1.   ,   0.   ]])